# <center> Warm Up

1)

In [ ]:
class Televisao:
    def __init__(self, ligada=False, canal=2):
        self.ligada = ligada
        self.canal = canal


2)

In [2]:
class Televisao:
    def __init__(self, tamanho, marca, ligada=False, canal=2):
        self.tamanho = tamanho
        self.marca = marca
        self.ligada = ligada
        self.canal = canal

tv = Televisao(55, "samsung")

outra_tv = Televisao(40, "LG")

print(tv.tamanho, tv.marca)
print(outra_tv.tamanho, outra_tv.marca)

55 samsung
40 LG


3)

In [ ]:
class Televisao:
    def __init__(self, tamanho, marca, ligada=False, canal=2):
        self.tamanho = tamanho
        self.marca = marca
        self.ligada = ligada
        self.canal = canal
    
    def muda_canal_para_cima(self):
        self.canal += 1

    def muda_canal_para_baixo(self):
        self.canal -= 1

tv = Televisao(55, "samsung")

outra_tv = Televisao(40, "LG")

print(tv.tamanho, tv.marca)
print(outra_tv.tamanho, outra_tv.marca)

# <center> Exercícios

## Exercício 1: Banco

In [1]:
from typing import Dict, List, Tuple


class Cliente:
    def __init__(self, nome: str, telefone: int):
        self.__nome = nome
        self.__telefone = telefone

    @property
    def nome(self):
        return self.__nome

    @property
    def telefone(self):
        return self.__telefone


class Conta:
    def __init__(self, titulares: Tuple[Cliente]):
        self.__titulares = titulares
        self.__saldo = 0

    @property
    def titulares(self):
        return self.__titulares

    @property
    def saldo(self):
        return self.__saldo

    def depositar(self, valor: float):
        self.__saldo += valor

    def sacar(self, valor: float):
        if valor > self.__saldo:
            raise ValueError("saldo insuficiente")

        self.__saldo -= valor


class ContaPoupanca(Conta):
    def __init__(self, *args, taxa=0, **kwargs):
        super().__init__(*args, **kwargs)

        self.__taxa_rendimento = taxa

    @property
    def taxa_rendimento(self):
        return self.__taxa_rendimento

    def render(self):
        self.depositar(self.saldo * self.__taxa_rendimento)


class Banco:
    def __init__(self, nome: str):
        self.__nome = nome

        self.__contas: List[Conta] = []
        self.__clientes: Dict[int, Cliente] = {}
        self.__taxa_rendimento = 0.065

    @property
    def nome(self):
        return self.__nome

    def cadastrar_cliente(self, nome: str, telefone: int):
        if nome in self.__clientes:
            raise ValueError(f"cliente '{nome}' já está cadastrado")

        cliente = Cliente(nome, telefone)
        self.__clientes[nome] = cliente

        return cliente

    def get_cliente(self, nome: str):
        return self.__clientes[nome]

    def abrir_conta(self, titulares: Tuple[Cliente], poupanca=False):
        if poupanca:
            conta = ContaPoupanca(titulares, taxa=self.__taxa_rendimento)
        else:
            conta = Conta(titulares)

        self.__contas.append(conta)

        return conta

    def get_contas(self, titular: Cliente):
        return tuple(conta for conta in self.__contas if titular in conta.titulares)

    def rendimento_mensal(self):
        for conta in self.__contas:
            if isinstance(conta, ContaPoupanca):
                conta.render()


if __name__ == "__main__":
    banco = Banco("Tatu")

    fulano = banco.cadastrar_cliente("Fulano de Tal", 48999887766)
    flavinho = banco.cadastrar_cliente("Flavinho do Pneu", 31415926535)

    try:
        banco.cadastrar_cliente("Fulano de Tal", 12312442112)
    except ValueError as e:
        print(e)

    conta = banco.abrir_conta((fulano, flavinho))

    print(conta.titulares)
    print(conta.saldo)

    try:
        conta.saldo = 1_000_000
    except AttributeError as e:
        print(e)

    conta.depositar(500)
    print(conta.saldo)

    conta.sacar(119.99)
    print(conta.saldo)

    try:
        conta.sacar(400)
    except ValueError as e:
        print(e)

    joao = banco.cadastrar_cliente("João Silva", 13987374627)
    poupanca_joao = banco.abrir_conta((joao,), poupanca=True)

    poupanca_joao.depositar(100)

    print(poupanca_joao.saldo)

    banco.rendimento_mensal()

    print(poupanca_joao.saldo)


    banco.abrir_conta((joao,))

    for conta in banco.get_contas(joao):
        print(f"{joao.nome} tem R${conta.saldo}")


cliente 'Fulano de Tal' já está cadastrado
(<__main__.Cliente object at 0x7f207c1a15f8>, <__main__.Cliente object at 0x7f207c1a16a0>)
0
can't set attribute
500
380.01
saldo insuficiente
100
106.5
João Silva tem R$106.5
João Silva tem R$0


## Exercício 3: Polinômio

In [9]:
from itertools import zip_longest
from typing import Any, Iterable, Tuple


class Polinomio:
    def __init__(self, *coefs: Tuple[float]):
        if isinstance(coefs[0],  Iterable):
            coefs = coefs[0]

        self.__coefs = tuple(coefs)

    @property
    def coefs(self):
        return self.__coefs

    @property
    def grau(self):
        return len(self.__coefs) - 1

    def evaluate(self, x: float):
        return sum(coef * x**i for i, coef in enumerate(self.__coefs))
    
    def plot(self, start: float, end: float, num: int):
        step = (end - start) / num

        return tuple(self.evaluate(start + step * i) for i in range(num))

    def __getitem__(self, potencia: int):
        return self.__coefs[potencia]
    
    def __call__(self, x: float):
        return self.evaluate(x)

    def __add__(self, value: Any):
        if isinstance(value, Polinomio):
            return Polinomio(
                a + b for a, b in zip_longest(self.coefs, value.coefs, fillvalue=0)
            )
    
    def __mul__(self, value: Any):
        if isinstance(value, Polinomio):
            size = len(self.coefs) + len(value.coefs) - 1
            coefs = [0] * size

            for i, a in enumerate(self.coefs):
                for j, b in enumerate(value.coefs):
                    coefs[i + j] += a * b
            
            return Polinomio(coefs)

        if isinstance(value, (int, float)):
            return Polinomio(
                coef * value for coef in self.__coefs
            )
    
    def __rmul__(self, value: Any):
        return self.__mul__(value)
    
    def __eq__(self, value: Any):
        if isinstance(value, Polinomio):
            return self.coefs == value.coefs
    
    def __repr__(self):
        return f"Polinomio(coefs={self.coefs})"


if __name__ == "__main__":
    quad = Polinomio(10, 5, 2)  # 2x^2 + 5x + 10

    print(quad)

    print(quad.coefs)
    print(quad.grau)

    print(quad[2], quad[1], quad[0])

    print(quad.evaluate(0))
    print(quad(0))

    cubic = Polinomio(4, 2, 1, 0.5) # (1/2)x^3 + x^2 + 2x + 4
    print(cubic)

    print(cubic(0))
    print(cubic(2))

    print(quad + cubic)

    t = Polinomio(1, 1) * Polinomio(1, -1) # (1 + x)(1 - x)
    print(t)
    print(t == Polinomio(1, 0, -1)) # (1 + x)(1 - x) = 1 - x²
    
    print(cubic.plot(-10, 10, 20))

Polinomio(coefs=(10, 5, 2))
(10, 5, 2)
2
2 5 10
10
10
Polinomio(coefs=(4, 2, 1, 0.5))
4.0
16.0
Polinomio(coefs=(14, 7, 3, 0.5))
Polinomio(coefs=(1, 0, -1))
True
(-416.0, -297.5, -204.0, -132.5, -80.0, -43.5, -20.0, -6.5, 0.0, 2.5, 4.0, 7.5, 16.0, 32.5, 60.0, 101.5, 160.0, 238.5, 340.0, 467.5)


## Exercício 4: Séries

In [1]:
class Math:
    def __init__(self):
        self.__fib_cache = {}
        self.__fact_cache = {}

    def fibonacci(self, n: int):
        if n <= 1:
            return 1

        if n not in self.__fib_cache:
            self.__fib_cache[n] = self.fibonacci(n-1) + self.fibonacci(n-2)
            
        return self.__fib_cache[n]
    
    def fatorial(self, n: int):
        if n <= 1:
            return 1
        
        if n not in self.__fact_cache:
            self.__fact_cache[n] = n * self.fatorial(n-1)
        
        return self.__fact_cache[n]

    def primos(self, n: int = float("inf")):
        """Gera os n primeiros números primos."""

        seq = [2]
        i = 3

        yield 2

        while True:
            if not any(i % p == 0 for p in seq):
                seq.append(i)
                yield i

                if len(seq) >= n:
                    break

            i += 2


if __name__ == "__main__":
    math = Math()

    print(math.fibonacci(20))

    print([math.fibonacci(i) for i in range(20)])

    print([math.fatorial(i) for i in range(10)])

    print(*math.primos(40), sep=" ")

10946
[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765]
[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880]
2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97 101 103 107 109 113 127 131 137 139 149 151 157 163 167 173
